# Creating my first graph and running it in a session

The following code creates the graph

In [1]:
import tensorflow as tf

x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

This code does not actually perform any computation. In fact, even the variables are not initialized yet. To evaluate this graph, you need to open a TensorFlow _session_ and use it to initialize the variables and evaluate **f**.

In [2]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print (result)
sess.close()

42


Having to repeat `sess.run()` all the time is a bit cumbersome. Here is the better way.

In [3]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

Instead of manually running the initializer for every single variable, you can use the `global_variables_initializer()` function

In [4]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run() # actually initializer all the variables
    result = f.eval()

Inside Jupyter, you may prefer to create an `InteractiveSession`. It automatically sets itself as the default session, so you don't need a with block ( but need to close the session manually)

In [5]:
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
sess.close()

42


#### A TensorFlow program is typically split into two parts: 

1. Building a computation graph, called the *construction phase*
2. Running it, *the execution phase*

In [6]:
import tensorflow as tf

# The first part: the construction phase
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

init = tf.global_variables_initializer()

# The second part: the execution phase
with tf.Session() as sess:
    init.run() 
    result = f.eval()
    print(result)

42


# Managing Graphs

Any node you create is automatically added to the default graph:

In [7]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

You can manage multiple independent graphs by creating a new `Graph` and temporarily making it the default graph inside a `with` block, like so:

In [8]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

In [9]:
x2.graph is graph

True

In [10]:
x2.graph is tf.get_default_graph

False

# Lifecycle of a Node Value

A variable starts its life when its initializer is run, and it ends when the session is closed.

In [11]:
w = tf.constant(3)
x = w + 2
y = x + 5 # y depends on x, which depends on w:  w -> x -> y
z = x * 3 

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


To evaluate y and z efficiently, w/o evaluating w and x twice as in the previous code, you must ask TensorFlow to evaluate both y and z in just one graph run, as shown in the following code:

In [12]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

10
15


# Linear Regression with Tensorflow

In [13]:
# It starts by fetching the dataset
import tensorflow as tf
import numpy as np
from sklearn.datasets import fetch_california_housing

# Adding an extra bias input feature (x0 = 1) to all training instances
housing = fetch_california_housing()
m, n = housing.data.shape # (20640, 8)
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data] # (20640, 9)

# Creating two TensorFlow constant nodes, X and y
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X") # (20640, 9)
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y') # (20640, 1)
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y) # Review: The Normal Equation

with tf.Session() as sess:
    theta_value = theta.eval()

### Review: The Normal Equation

A mathematical equation that gives the result directly

$$ \hat{\theta} = (\mathbf{X^T} \cdot  \mathbf{X})^{-1} \cdot \mathbf{X^T} \cdot y $$

* $\hat{\theta}$ is the value of $\theta$ that minimizes the cost function.
* $y$ is the vector of target values containing $y^{(1)}$ to $y^{(m)}$

### NumPy vs. TensorFlow: The main benefit
TensorFlow will automatically run this on your GPU card

# Implementing Gradient Descent

Using Batch Gradient Descent instead of the Normal Equation, we will do this:

1. by manually computing the gradients
2. by using TensorFlow's autodiff feature
3. by using Tensorflow's out-of-the-box optimizers

**important notes: normalize the input feature vectors first**

## Manually Computing the Gradient

In [14]:
# Normalize the feature vector: scaled_housing_data_plus_bias
from sklearn import preprocessing

scaler =preprocessing.StandardScaler().fit(housing_data_plus_bias)

scaled_housing_data_plus_bias = scaler.transform(housing_data_plus_bias)

In [15]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta") # random_uniform() generating a tensor containing random values
y_pred = tf.matmul(X, theta, name="prediction")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()   
    

Epoch 0 MSE = 10.6591
Epoch 100 MSE = 5.04315
Epoch 200 MSE = 4.93309
Epoch 300 MSE = 4.90303
Epoch 400 MSE = 4.881
Epoch 500 MSE = 4.86416
Epoch 600 MSE = 4.85121
Epoch 700 MSE = 4.84119
Epoch 800 MSE = 4.83341
Epoch 900 MSE = 4.82733


# Using autodiff

Simply replcae the `gradients = ...` line in the Gradient Descent code in the previous section

In [16]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta") # random_uniform() generating a tensor containing random values
y_pred = tf.matmul(X, theta, name="prediction")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
#gradients = 2/m * tf.matmul(tf.transpose(X), error)
gradients = tf.gradients(mse, [theta])[0]
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()   
    

Epoch 0 MSE = 8.41548
Epoch 100 MSE = 4.8767
Epoch 200 MSE = 4.82769
Epoch 300 MSE = 4.81986
Epoch 400 MSE = 4.81557
Epoch 500 MSE = 4.8125
Epoch 600 MSE = 4.81023
Epoch 700 MSE = 4.80854
Epoch 800 MSE = 4.80728
Epoch 900 MSE = 4.80634


# Using an Optimizer
You can simply replace the preceding `gradients = ...` and `training_op = ...` lines

In [17]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta") # random_uniform() generating a tensor containing random values
y_pred = tf.matmul(X, theta, name="prediction")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
# gradients = 2/m * tf.matmul(tf.transpose(X), error)
# gradients = tf.gradients(mse, [theta])[0]
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
training_op = optimizer.minimize(mse)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()   
    

Epoch 0 MSE = 7.08913
Epoch 100 MSE = 52.0352
Epoch 200 MSE = 185.096
Epoch 300 MSE = 406.273
Epoch 400 MSE = 715.566
Epoch 500 MSE = 1112.98
Epoch 600 MSE = 1598.5
Epoch 700 MSE = 2172.14
Epoch 800 MSE = 2833.89
Epoch 900 MSE = 3583.76
